In [0]:
create or refresh streaming table bronze_orders
comment "this is bronze table"
tblproperties ("quality" = "true")
as
select *,_metadata.file_path as input_file_path,current_timestamp() as ingestion_time
from cloud_files("/Volumes/circuitbox/landing/operationaldata/orders/","json",map("cloudFiles.inferColumnTypes" , "true"))

In [0]:
create or refresh streaming table silver_orders_clean(
  constraint validate_customer_id expect(customer_id is not null) on violation fail update,
  constraint validate_order_id expect(order_id is not null) on violation fail update,
  constraint validate_order_status expect(order_status in ('Pending','Completed','Shipped','Cancelled')),
  constraint validate_payment_method expect(payment_method in ('Credit Card','PayPal','Bank Transfer'))
)
as
select 
   customer_id,
   order_id,
   order_status,
 order_timestamp,
   payment_method,
   items.category as category,
   items.item_id as item_id,
   items.price as price,
   items.name as name,
   items.quantity as quantity
from(
  select 
  customer_id,
   order_id,
   order_status,
   cast(order_timestamp as timestamp) as order_timestamp,
   payment_method,
   explode(items) as items
   from stream(live.bronze_orders)
)